<a href="https://colab.research.google.com/github/jaeohshin/ML_with_Pytorch_Sklearn_rasbt/blob/main/Car_Mileage_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

from IPython.display import Image

In [2]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'


In [3]:
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']
df = pd.read_csv(url, names=column_names,
                 na_values = "?", comment='\t',
                 sep=" ", skipinitialspace=True)

df.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [4]:
print(df.isna().sum())

df = df.dropna()
df = df.reset_index(drop=True)
df.tail()

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64


,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
387,27.0,4,140.0,86.0,2790.0,15.6,82,1
388,44.0,4,97.0,52.0,2130.0,24.6,82,2
389,32.0,4,135.0,84.0,2295.0,11.6,82,1
390,28.0,4,120.0,79.0,2625.0,18.6,82,1
391,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [5]:
print(df.isna().sum())

MPG             0
Cylinders       0
Displacement    0
Horsepower      0
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64


In [6]:
import sklearn
import sklearn.model_selection

In [9]:
df_train, df_test = sklearn.model_selection.train_test_split(df, train_size=0.7, random_state=1)
train_stats = df_train.describe().transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
MPG,274.0,23.413869,7.649130,9.0,17.5,23.0,28.775,46.6
Cylinders,274.0,5.383212,1.706229,3.0,4.0,4.0,7.500,8.0
Displacement,274.0,188.994526,102.744326,68.0,105.0,140.0,259.500,455.0
Horsepower,274.0,103.010949,37.873946,46.0,75.0,92.5,123.750,230.0
Weight,274.0,2956.536496,846.378383,1613.0,2220.0,2750.0,3568.250,5140.0
Acceleration,274.0,15.694891,2.764859,8.5,14.0,15.5,17.300,24.8
Model Year,274.0,75.857664,3.695086,70.0,73.0,76.0,79.000,82.0
Origin,274.0,1.591241,0.803424,1.0,1.0,1.0,2.000,3.0


In [10]:
numeric_column_names = [
    'Cylinders', 'Displacement',
    'Horsepower', 'Weight',
    'Acceleration'
]

In [13]:
df_train_norm, df_test_norm = df_train.copy(), df_test.copy()
for col_name in numeric_column_names:
    mean = train_stats.loc[col_name]['mean']
    std = train_stats.loc[col_name]['std']
    df_train_norm.loc[:,col_name] = (df_train_norm.loc[:,col_name] - mean) / std
    df_test_norm.loc[:,col_name] = (df_test_norm.loc[:,col_name] - mean) / std

df_train_norm.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
203,28.0,-0.810683,-0.895373,-0.739584,-0.947019,0.255026,76,3
255,19.4,0.361492,0.418568,-0.343533,0.299468,0.544371,78,1
72,13.0,1.533667,1.148535,0.712602,1.348644,-0.613012,72,1
235,30.5,-0.810683,-0.885640,-1.056424,-1.069896,0.472035,77,1
37,14.0,1.533667,1.567050,1.636720,1.479791,-1.336376,71,1


In [14]:
boundaries = torch.tensor([73, 76, 79])

v = torch.tensor(df_train_norm['Model Year'].values)
df_train_norm['Model Year Bucketed'] = torch.bucketize(v, boundaries, right=True)

v = torch.tensor(df_test_norm['Model Year'].values)
df_test_norm['Model Year Bucketed'] = torch.bucketize(v, boundaries, right=True)

numeric_column_names.append('Model Year Bucketed')

In [16]:
from torch.nn.functional import one_hot

total_origin = len(set(df_train_norm['Origin']))

origin_encoded = one_hot(torch.from_numpy(df_train_norm['Origin'].values) % total_origin)
x_train_numeric = torch.tensor(df_train_norm[numeric_column_names].values)
x_train = torch.cat([x_train_numeric, origin_encoded], 1).float()

origin_encoded = one_hot(torch.from_numpy(df_test_norm['Origin'].values) % total_origin)
x_test_numeric = torch.tensor(df_test_norm[numeric_column_names].values)
x_test = torch.cat([x_test_numeric, origin_encoded], 1).float()

In [17]:
y_train = torch.tensor(df_train_norm['MPG'].values).float()
y_test = torch.tensor(df_test_norm['MPG'].values).float()


In [30]:
x_test[0,:]

tensor([-0.8107, -0.6715, -0.1587, -0.5323, -0.4322,  0.0000,  1.0000,  0.0000,
         0.0000])